In [165]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
np.random.seed(23)

In [166]:
X, Y = make_classification(n_samples=10000, n_classes=2, n_informative=2, n_features=4)
X = X.reshape(X.shape[1], X.shape[0])
Y = Y.reshape(1, -1)
X = (X - np.mean(X, axis=1, keepdims=True)) / np.std(X, axis=1, keepdims=True)

In [167]:
def forward_pass(x, theta):
    J = x * theta
    return J

In [168]:
def backward_pass(x):
    dtheta = x
    return dtheta

In [169]:
def gradient_check(x, theta, eps=1e-7, print_msg=False):

    theta_plus = theta + eps
    theta_minus = theta - eps
    J_plus = forward_pass(x, theta_plus)
    J_minus = forward_pass(x, theta_minus)
    gradapprox = (J_plus - J_minus) / (2 * eps)

    grad = backward_pass(x)

    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [170]:
x, theta = 3, 4
difference = gradient_check(x, theta, print_msg=True)
gradient_check(x, theta)

Your backward propagation works perfectly fine! difference = 7.814075313343006e-11


7.814075313343006e-11

In [171]:
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

In [172]:
def relu(Z):
    return np.maximum(0.0, Z)

In [173]:
def initialize_params_deep(layer_dims):
    params = {}
    L = len(layer_dims)
    for l in range(1, L):
        params['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        params['b' + str(l)] = np.random.randn(layer_dims[l], 1)

    return params

In [174]:
layers_dims = [4, 3, 2, 1]

In [232]:
def forward_propagation(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    log_probs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(log_probs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [233]:
def backward_propagation(X, Y, cache):
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T)
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [234]:
def dictionary_to_vector(params):
    theta = []
    L = len(params) // 2
    for i in range(1, L + 1):
        w_flat = params['W' + str(i)].flatten().reshape(-1, 1)
        theta.append(w_flat)
        b_flat = params['b' + str(i)].flatten().reshape(-1, 1)
        theta.append(b_flat)

    theta = np.concatenate(theta, axis=0)

    return theta

In [235]:
def gradient_to_vector(grads):
    dtheta = []
    L = len(grads) // 2
    for i in range(1, L - 1):
        w_flat = grads['dW' + str(i)].flatten().reshape(-1, 1)
        dtheta.append(w_flat)
        b_flat = grads['db' + str(i)].flatten().reshape(-1, 1)
        dtheta.append(b_flat)

    dtheta = np.concatenate(dtheta, axis=0)

    return dtheta

In [236]:
def vector_to_dictionary(theta, layer_dims):
    params = {}
    idx = 0
    L = len(layer_dims)
    for i in range(1, L):
        w_size = layer_dims[i] * layer_dims[i - 1]
        params['W' + str(i)] = theta[idx: idx + w_size].reshape(layer_dims[i], layer_dims[i - 1])

        idx += w_size

        b_size = layer_dims[i]
        params['b' + str(i)] = theta[idx: idx + b_size].reshape(-1, 1)

        idx += b_size

    return params

In [237]:
params = initialize_params_deep(layers_dims)
theta = dictionary_to_vector(params)
params = vector_to_dictionary(theta, layers_dims)
print(params)

{'W1': array([[-0.00110136, -0.01105749, -0.01041781,  0.01057207],
       [-0.02095097,  0.01282701,  0.01003802,  0.00544912],
       [ 0.01159185, -0.00732564,  0.00960003, -0.0012885 ]]), 'b1': array([[-0.47022175],
       [-0.34316199],
       [-0.24312412]]), 'W2': array([[-0.00368929,  0.00837023,  0.01586876],
       [-0.00506199, -0.01639457, -0.02067641]]), 'b2': array([[-0.13775803],
       [ 1.98036322]]), 'W3': array([[ 0.00742434, -0.01750774]]), 'b3': array([[-1.17089968]])}


In [230]:
def gradient_check_deep(params, grads, X, Y, layers_dims, eps=1e-7):
    
    param_values = dictionary_to_vector(params)
    grad = gradient_to_vector(grads)
    
    num_parameters = param_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))

    for i in range(num_parameters):
        theta_plus = np.copy(param_values)
        theta_plus[i] += eps
        J_plus[i], _ = forward_propagation(X, Y, vector_to_dictionary(theta_plus, layers_dims))

        theta_minus = np.copy(param_values)
        theta_minus[i] -= eps
        J_minus[i], _ = forward_propagation(X, Y, vector_to_dictionary(theta_minus, layers_dims))

        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * eps)
    
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")

    return difference


In [231]:
params = initialize_params_deep(layers_dims)
cost, cache = forward_propagation(X, Y, params)
grads = backward_propagation(X, Y, cache)

gradient_check_deep(params, grads, X, Y, layers_dims)

Your backward propagation works perfectly fine! difference = 3.1307351510496765e-09


3.1307351510496765e-09